Don't forget to enable "AI Platform Training & Prediction API" API: https://console.cloud.google.com/apis/api/ml.googleapis.com/metrics?project=YOUR-PROJECT-ID

Obtaining OAuth 2 Token for a non interactive use: https://saturncloud.io/blog/authenticate-to-google-container-service-with-script-noninteractive-gcloud-auth-login/

Execute these in a shell:
1. Authenticate to GCS: `gcloud auth login --brief --quiet`
2. Retrieve refresh token: `REFRESH_TOKEN=$(gcloud auth print-access-token)`
3. Activate refresh token: `gcloud auth activate-refresh-token $REFRESH_TOKEN`

In [13]:
import os
import base64
import requests
from io import BytesIO
from base64 import b64encode
import urllib.request

def predict_api(image_file, bearer_token, parameters=None):
    ENCODING = "utf-8"
    # 1. Reading the binary stuff
    # note the 'rb' flag
    # result: Bytes
    IMAGE_NAME = image_file

    with open(IMAGE_NAME, "rb") as open_file:
        byte_content = open_file.read()

    # 2. Base64 encode read data
    # result: bytes (again)
    base64_bytes = b64encode(byte_content)

    # 3. Decode these bytes to text
    # result: string (in utf-8)
    base64_string = base64_bytes.decode(ENCODING)

    VQA_PROMPT = "Describe the content of the image in great detail"

    payload = {
      "instances": [
        {
          "prompt": VQA_PROMPT,
          "image": {
              "bytesBase64Encoded": base64_string
          }
        }
      ],
      "parameters": parameters
    }

    url = "https://us-central1-aiplatform.googleapis.com/v1/projects/gdg-demos/locations/us-central1/publishers/google/models/imagetext:predict"
    headers = {
        "Authorization": "Bearer {}".format(bearer_token),
        "Accept": "application/json; charset=utf-8",
    }
    json_data = requests.post(url, headers=headers, json=payload) #, verify=False)
    return json_data

In [22]:
# download file
# image_file_url = ""

image_file = "pexels-soly-moses-5897400.jpg"
# urllib.request.urlretrieve(image_file_url, "test_image.jpg")

RESPONSE_COUNT = 1

parameters = {
    "sampleCount": RESPONSE_COUNT
}

# Replace with your own token:
# Execute `gcloud auth print-access-token` in a cloud shell to get the Authorization Bearer token
gcp_bearer_token = os.getenv("GCP_BEARER_TOKEN") or "GCP_BEARER_TOKEN"
return_val = predict_api(image_file, gcp_bearer_token, parameters=parameters)
print(return_val.content)

<Response [403]>
